In [20]:
import os
import shutil
import pandas as pd
from geopy.distance import geodesic
import pandas as pd
import folium


In [6]:
# get common data files from both years and copy them to the destination folders

def get_common_csv_files(folder1, folder2):
    files_folder1 = set(file for file in os.listdir(folder1) if file.endswith('.csv'))
    files_folder2 = set(file for file in os.listdir(folder2) if file.endswith('.csv'))
    
    common_files = files_folder1.intersection(files_folder2)
    return common_files

def copy_files(source_folder, destination_folder, files_to_copy):
    for file in files_to_copy:
        source_path = os.path.join(source_folder, file)
        destination_path = os.path.join(destination_folder, file)
        shutil.copy2(source_path, destination_path)

# Replace these paths with your actual paths
folder1_path = 'E:/xie/Sensor Files/4. impute/full MICE imputated/2018'
folder2_path = 'E:/xie/Sensor Files/4. impute/full MICE imputated/2019'
destination_folder1_path = 'E:/xie/Sensor Files/5. dataset/full MICE imputed datasets/2018'
destination_folder2_path = 'E:/xie/Sensor Files/5. dataset/full MICE imputed datasets/2019'

common_files = get_common_csv_files(folder1_path, folder2_path)

# Copy common files to destination folders
copy_files(folder1_path, destination_folder1_path, common_files)
copy_files(folder2_path, destination_folder2_path, common_files)

print("Common CSV files copied successfully.")


Common CSV files copied successfully.


In [21]:
# get number of sensor present in one region
# visualize sensor locations on a map for sanity check

def filter_detectors(data, location, ignore):
    if ignore is None:
        filtered_detectors = data[data['Location'].str.contains(location)]
    else:
        filtered_detectors = data[data['Location'].str.contains(location) & ~data['Location'].str.contains(ignore)]
    return filtered_detectors

def plot_map(data):
    # Extract latitude, longitude, detector ID, and location for I-15 detectors
    latitude = data["Latitude"]
    longitude = data["Longitude"]
    detector_id = data["DetectorID"]
    location = data["Location"]

    # Create a map centered at the mean latitude and longitude, with zoom level 10
    map = folium.Map(location=[latitude.mean(), longitude.mean()], zoom_start=10)

    # Add markers for each detector location
    for lat, lon, det_id, loc in zip(latitude, longitude, detector_id, location):
        tooltip_text = f"Detector ID: {det_id}<br>Location: {loc}<br>Latitude: {lat}<br>Longitude: {lon}"
        folium.CircleMarker(location=[lat, lon], radius=5, color='blue', fill=True, fill_color='blue',
                            tooltip=tooltip_text).add_to(map)

    # Display the map
    display(map)

def main(csv_file, location, ignore):
    # Read the CSV file
    data = pd.read_csv(csv_file, delimiter=',')

    # Filter detectors based on location criteria and ignore patterns
    filtered_data = filter_detectors(data, location, ignore)

    # Count the number of sensors found
    num_sensors = len(filtered_data)
    print(f"Number of sensors found in {location}: {num_sensors}")

    # Plot map for filtered detectors
    plot_map(filtered_data)


if __name__ == "__main__":
    csv_file = 'E:/xie/common_sensor_data_ignr1.csv'  # Provide the path to your CSV file containing detector information
    location = 'I-15 NB'  # Location criteria
    #ignore= None
    ignore = 'OnRamp|OffRamp|Access'  # Patterns to ignore
    main(csv_file, location, ignore)


Number of sensors found in I-15 NB: 63


In [12]:
# trying out new algo # train and val/test dataset generation

def filter_detectors(data, location, ignore):
    if ignore is None:
        filtered_detectors = data[data['Location'].str.contains(location)]
    else:
        filtered_detectors = data[data['Location'].str.contains(location) & ~data['Location'].str.contains(ignore)]
    return filtered_detectors

def find_closest_sensor(sensor_data, start_sensor_lat, start_sensor_lon, visited):
    closest_sensor_id = None
    min_distance = float('inf')

    for index, sensor in sensor_data.iterrows():
        sensor_id = sensor['DetectorID']
        lat = sensor['Latitude']
        lon = sensor['Longitude']

        if sensor_id not in visited:
            distance = geodesic((start_sensor_lat, start_sensor_lon), (lat, lon)).kilometers
            if distance < min_distance:
                min_distance = distance
                closest_sensor_id = sensor_id

    return closest_sensor_id, min_distance

def tsp(sensor_data, start_sensor_id):
    visited = set()
    visited.add(start_sensor_id)

    # Print the start_sensor_id
    print(f"Starting from sensor ID: {start_sensor_id}")

    # Count the number of sensors found
    num_sensors = 1

    while len(visited) < len(sensor_data):
        # Find the coordinates of the current sensor
        current_sensor = sensor_data[sensor_data['DetectorID'] == start_sensor_id].iloc[0]
        current_lat = current_sensor['Latitude']
        current_lon = current_sensor['Longitude']

        # Find the closest sensor to the current sensor
        closest_sensor_id, min_distance = find_closest_sensor(sensor_data, current_lat, current_lon, visited)

        # Print the closest sensor
        print(f"Detector ID: {closest_sensor_id}")

        # Update the start_sensor_id for the next iteration
        start_sensor_id = closest_sensor_id
        visited.add(start_sensor_id)

        # Increment the number of sensors found
        num_sensors += 1

    print(f"Total number of sensors found: {num_sensors}")

if __name__ == "__main__":
    csv_file = 'E:/xie/common_sensor_data_ignr1.csv'  # Provide the path to your CSV file containing detector information
    location = 'I-15 NB'  # Location criteria
    ignore = 'OnRamp|OffRamp|Access'  # Patterns to ignore
    
    # Read the CSV file
    data = pd.read_csv(csv_file, delimiter=',')
    
    # Filter detectors based on location criteria and ignore patterns
    filtered_data = filter_detectors(data, location, ignore)
    
    # Choose a start sensor (you can change this to any specific DetectorID)
    start_sensor_id = '450_2_359'  # Example starting sensor ID
    
    # Call the tsp function to print out sensors starting from the start sensor if it's present in the filtered group
    tsp(filtered_data, start_sensor_id)


Starting from sensor ID: 450_2_359
Detector ID: 449_2_357
Detector ID: 448_1_355
Detector ID: 448_2_354
Detector ID: 448_3_353
Detector ID: 446_1_350
Detector ID: 446_3_349
Detector ID: 445_1_348
Detector ID: 445_3_346
Detector ID: 444_3_345
Detector ID: 443_1_344
Detector ID: 443_2_343
Detector ID: 443_3_342
Detector ID: 442_2_340
Detector ID: 441_1_338
Detector ID: 441_3_336
Detector ID: 439_2_333
Detector ID: 438_1_331
Detector ID: 438_2_330
Detector ID: 437_2_328
Detector ID: 437_3_327
Detector ID: 357_3_310
Detector ID: 356_1_309
Detector ID: 356_2_308
Detector ID: 355_1_156
Detector ID: 355_2_153
Detector ID: 355_3_155
Detector ID: 354_3_145
Detector ID: 34_1_94
Detector ID: 48_2_83
Detector ID: 49_1_82
Detector ID: 49_2_12
Detector ID: 49_3_15
Detector ID: 58_2_17
Detector ID: 59_2_18
Detector ID: 72_2_28
Detector ID: 89_1_28
Detector ID: 89_2_30
Detector ID: 97_3_38
Detector ID: 146_2_238
Detector ID: 149_2_240
Detector ID: 160_2_242
Detector ID: 396_1_243
Detector ID: 396_2_24

In [26]:
# algo works fine !


import pandas as pd
import os
from geopy.distance import geodesic

def filter_detectors(data, location, ignore):
    if ignore is None:
        filtered_detectors = data[data['Location'].str.contains(location)]
    else:
        filtered_detectors = data[data['Location'].str.contains(location) & ~data['Location'].str.contains(ignore)]
    return filtered_detectors

def tsp(sensor_data, start_sensor_id, sensor_directory, output_directory):
    visited = set()
    visited.add(start_sensor_id)

    # Count the number of sensors found
    num_sensors = 1

    # Create an empty DataFrame to store speeds
    speed_df = pd.DataFrame()

    while len(visited) <= len(sensor_data):
        # Find the current sensor
        current_sensor = sensor_data[sensor_data['DetectorID'] == start_sensor_id].iloc[0]

        # Print the current sensor
        print(f"Current Detector ID: {start_sensor_id}")

        # Load speed data for the current sensor
        file_name = start_sensor_id.replace('_', '.')
        file_path = os.path.join(sensor_directory, f"{file_name}_impute.csv")
        speed_data = pd.read_csv(file_path)

        # Concatenate speed column
        speed_df = pd.concat([speed_df, speed_data['Speed']], axis=1)

        # Find the closest sensor to the current sensor
        min_distance = float('inf')
        closest_sensor_id = None
        current_lat = current_sensor['Latitude']
        current_lon = current_sensor['Longitude']
        
        for index, sensor in sensor_data.iterrows():
            sensor_id = sensor['DetectorID']
            lat = sensor['Latitude']
            lon = sensor['Longitude']

            if sensor_id not in visited:
                distance = geodesic((current_lat, current_lon), (lat, lon)).kilometers
                if distance < min_distance:
                    min_distance = distance
                    closest_sensor_id = sensor_id

        # Update the start_sensor_id for the next iteration
        start_sensor_id = closest_sensor_id
        visited.add(start_sensor_id)

        # Increment the number of sensors found
        num_sensors += 1

    print(f"Total number of sensors found: {num_sensors-1}")

    # Save the concatenated speed data to a CSV file
    output_file = os.path.join(output_directory, "val_test.csv")
    speed_df.to_csv(output_file, index=False, header=False)
    print(f"Data saved to {output_file}.csv")

if __name__ == "__main__":
    csv_file = 'E:/xie/common_sensor_data_ignr1.csv'  # Provide the path to your CSV file containing detector information
    location = 'I-15 NB'  # Location criteria
    ignore = 'OnRamp|OffRamp|Access'  # Patterns to ignore
    sensor_directory = 'E:/xie/Sensor Files/4. impute/full MICE imputed Common Files/2019'  # Directory containing sensor files
    output_directory = 'E:/xie/Sensor Files/5. dataset/full mice impute/I15 NB ignore all'  # Directory to save concatenated speed data
    
    # Read the CSV file
    data = pd.read_csv(csv_file, delimiter=',')
    
    # Filter detectors based on location criteria and ignore patterns
    filtered_data = filter_detectors(data, location, ignore)
    
    # Choose a start sensor (you can change this to any specific DetectorID)
    start_sensor_id = '450_2_359'  # Example starting sensor ID
    
    # Call the tsp function to concatenate speed data from sensors starting from the start sensor
    tsp(filtered_data, start_sensor_id, sensor_directory, output_directory)


Current Detector ID: 450_2_359
Current Detector ID: 449_2_357
Current Detector ID: 448_1_355
Current Detector ID: 448_2_354
Current Detector ID: 448_3_353
Current Detector ID: 446_1_350
Current Detector ID: 446_3_349
Current Detector ID: 445_1_348
Current Detector ID: 445_3_346
Current Detector ID: 444_3_345
Current Detector ID: 443_1_344
Current Detector ID: 443_2_343
Current Detector ID: 443_3_342
Current Detector ID: 442_2_340
Current Detector ID: 441_1_338
Current Detector ID: 441_3_336
Current Detector ID: 439_2_333
Current Detector ID: 438_1_331
Current Detector ID: 438_2_330
Current Detector ID: 437_2_328
Current Detector ID: 437_3_327
Current Detector ID: 357_3_310
Current Detector ID: 356_1_309
Current Detector ID: 356_2_308
Current Detector ID: 355_1_156
Current Detector ID: 355_2_153
Current Detector ID: 355_3_155
Current Detector ID: 354_3_145
Current Detector ID: 34_1_94
Current Detector ID: 48_2_83
Current Detector ID: 49_1_82
Current Detector ID: 49_2_12
Current Detector

In [25]:
# edge list generation

def create_edge_list(sensor_data):
    num_sensors = len(sensor_data)
    edge_list = []

    for i in range(1, num_sensors + 1):
        for j in range(1, num_sensors + 1):
            if i == j:
                continue
            if abs(i - j) == 1:
                weight = 1  # Nodes i and j are adjacent, so weight is 1
            else:
                weight = 0  # Nodes i and j are not adjacent, so weight is 0
            edge_list.append([i, j, weight])

    return edge_list

def save_edge_list(edge_list, filename):
    edge_df = pd.DataFrame(edge_list, columns=['sensor1', 'sensor2', 'weight'])
    edge_df.to_csv(filename, index=False, sep='\t')

if __name__ == "__main__":
    csv_file = 'E:/xie/common_sensor_data_ignr1.csv'  # Provide the path to your CSV file containing detector information
    location = 'I-15 NB'  # Location criteria
    ignore = 'OnRamp|OffRamp|Access'  # Patterns to ignore
    
    # Read the CSV file
    data = pd.read_csv(csv_file, delimiter=',')
    
    # Filter detectors based on location criteria and ignore patterns
    filtered_data = filter_detectors(data, location, ignore)
    
    # Create the edge list
    edge_list = create_edge_list(filtered_data)
    
    # Formulate the edge list file name
    edge_list_filename = f'{location.replace(" ", "_")}_edge_list.txt'
    
    # Specify the location to save the edge list file
    save_location = 'E:/xie/Sensor Files/5. dataset/full mice impute/I15 NB ignore all'
    
    # Save the edge list as a text file
    save_edge_list(edge_list, os.path.join(save_location, edge_list_filename))

In [36]:
# spatial embedding generation
from node2vec import node2vec
import numpy as np
import networkx as nx
from gensim.models import Word2Vec

is_directed = True
p = 2
q = 1
num_walks = 100
walk_length = 80
dimensions = 64
window_size = 10
iter = 1000
Adj_file = 'E:/xie/Sensor Files/5. dataset/full mice impute/I15 NB ignore all/I-15_NB_edge_list.txt'
SE_file = 'E:/xie/Sensor Files/5. dataset/full mice impute/I15 NB ignore all/I-15_NB_SE.txt'

def read_graph(edgelist):
    G = nx.read_edgelist(
        edgelist, nodetype=int, data=(('weight',float),),
        create_using=nx.DiGraph())

    return G

def learn_embeddings(walks, dimensions, output_file):
    walks = [list(map(str, walk)) for walk in walks]
    model = Word2Vec(
        walks, vector_size = dimensions, window = 10, min_count=0, sg=1,
        workers = 8, epochs = iter)
    model.wv.save_word2vec_format(output_file)
	
    return
    
nx_G = read_graph(Adj_file)
G = node2vec.Graph(nx_G, is_directed, p, q)
G.preprocess_transition_probs()
walks = G.simulate_walks(num_walks, walk_length)
learn_embeddings(walks, dimensions, SE_file)
print('Generating SE...')
print('SE file saved!')


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations